In [72]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier

import numpy as np


batch_size = 128
num_classes = 10
epochs = 12

# input image dimensions
img_rows, img_cols = 28, 28

# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [66]:
global_data = x_train
global_labels = y_train

x_test = x_train[1000:2000]
y_test = y_train[1000:2000]
x_train = x_train[:1000]
y_train = y_train[:1000]

In [69]:
def data_input_simulator(x_train, y_train, x_test, y_test, global_data, global_labels, step_size = 1000):
    """
    This method simulates getting more data in the wild.  It takes a set of training data
    and testing data creates a new training set by combining the training and test set
    and then sampling a new test set from the unused samples in global_data
    """
    
    # Combine the test set into the training set
    new_train_data = np.concatenate([x_train, x_test], axis=0)
    new_train_labels = np.concatenate([y_train, y_test], axis=0)
    
    new_num_samples = new_train_data.shape[0]
    
    # Extract a new test set from the global data
    new_test_data = global_data[new_num_samples:new_num_samples + step_size]
    new_test_labels = global_labels[new_num_samples:new_num_samples + step_size]
    
    return new_train_data, new_test_data, new_train_labels, new_test_labels

In [70]:
a, b, c, d = data_input_simulator(x_train, y_train, x_test, y_test, global_data, global_labels)

In [32]:
# Define Transformers
class FlattenTransformer(BaseEstimator, TransformerMixin):
    """
    Flatten an existing array into 
    """

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        return X.reshape(X.shape[0], X.shape[1] * X.shape[2]).astype('float32') / 255
    
class ChannelTransformer(BaseEstimator, TransformerMixin):
    """
    Flatten an existing array into 
    """

    def __init__(self, img_rows, img_columns, img_channels, channels_first=False):
        self.img_rows = img_rows
        self.img_columns = img_columns
        self.img_channels = img_channels
        self.channels_first = channels_first

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        if self.channels_first:
            return X.reshape(X.shape[0], 1, self.img_rows, self.img_cols).astype('float32') / 255
        else:
            return X.reshape(X.shape[0], self.img_rows, self.img_columns, self.img_channels).astype('float32') / 255

In [33]:
# TODO:: Define NN based estimators

In [73]:
# Define an SVM Pipeline
svm_pipeline = Pipeline([
        ('flatten', FlattenTransformer()),
        ('classifier', svm.SVC(C=1, kernel='linear'))
    ])

In [74]:
# Defin a Random Forest Pipeline
rf_pipeline = Pipeline([
        ('flatten', FlattenTransformer()),
        ('classifier', RandomForestClassifier(n_estimators=25))
    ])

In [78]:
class model_staging_area():
    
    def __init__(self, models=None):
        self.models = [] if not models else models
        self.best_model = None
        
    def time_step_model_selection(self):
        """Retrains models and reevaluates them choosing a production model"""
        pass
    
    def evaluate_models(self):
        """Evaluates all models and returns the best one"""
        pass
    
    def predict(self, X):
        """Uses the best model to predict on the data"""
        return self.best_model.predict(X)